In [1]:
#Import libraries
import os
import geopandas as gp
import matplotlib.pyplot as plt
import json
from contextlib import redirect_stdout
import csv
import earthaccess


/home/atticus/miniconda3/envs/hls-linux/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Login
earthaccess.login(persist=True)

In [3]:
#Set time and location

field = gp.read_file(r'/mnt/c/Users/attic/HLS_Kelp/maps/Isla_Vista_Kelp.geojson')
bbox = tuple(list(field.total_bounds))
bbox #Display coordinate bounds
with open('/mnt/c/Users/attic/HLS_Kelp/maps/Isla_Vista_Kelp.geojson', 'r') as f:
    data = json.load(f)
# Extract the name
location = 'Isla_vista_kelp_2018'#(data['name']).replace('.kmz', '').replace(' ', '_')
temporal = ("2018-01-01T00:00:00", "2019-01-01T00:00:00") #
create_dem = False # set true if u want a digital elevation map 
dem_name = 'dem.tif'

num_to_download = 500 #set this value to the number of frames you want
load_num = -1 #sets number of granules to load, this should generally be >> than num_download;  -1 loads all 
specific_tile = False #set true if you only want specific tile 
tile = '10SGD' # 

In [4]:
#Search for satellite data from  Landsat 30m and Sentinel 30m
results = earthaccess.search_data(
    short_name=['HLSL30','HLSS30'],
    bounding_box=bbox,
    temporal=temporal,
     cloud_cover=0, #Determine cloud cover
    count=load_num
)
if create_dem:
    dem_results = earthaccess.search_data(
        short_name="ASTGTM",
        bounding_box=bbox)
    print(dem_results[0])

print(results[0])


Granules found: 183
Collection: {'EntryTitle': 'HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30m v2.0'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -120.25617766, 'Latitude': 34.20966703}, {'Longitude': -119.06537353, 'Latitude': 34.23552162}, {'Longitude': -119.09014717, 'Latitude': 35.2250277}, {'Longitude': -120.29518354, 'Latitude': 35.19820772}, {'Longitude': -120.25617766, 'Latitude': 34.20966703}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2018-01-02T18:54:21.450Z', 'EndingDateTime': '2018-01-02T18:54:21.450Z'}}
Size(MB): 220.48974514007568
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T11SKU.2018002T185421.v2.0/HLS.S30.T11SKU.2018002T185421.v2.0.B05.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T11SKU.2018002T185421.v2.0/HLS.S30.T11SKU.2018002T185421.v2.0.B03.tif', 'https://d

In [5]:
num_threads = os.cpu_count()
print(num_threads)

20


In [6]:

folder_path = os.path.join(os.getcwd(),(f'imagery/{location}'))
## ======= Create/write DEM, if requested ======= ##
if create_dem:
    if not os.path.isfile(os.path.join(folder_path, dem_name)):
            dem_path = earthaccess.download(dem_results[0], local_path=folder_path)
            os.rename(dem_path[0], os.path.join(folder_path,'dem.tif'))
            os.rename(dem_path[1], os.path.join(folder_path, 'num.tif'))

## ======= create location folder path ======= ##
if not os.path.isdir(folder_path):
    os.mkdir(folder_path)

downloaded = 0

for i, granule in enumerate(results):

    ## ======= Parse metadata ======= ##
    json_str = json.dumps(granule.__dict__)
    metadata = json.loads(json_str) 
    meta = metadata['render_dict']['meta']
    name = meta['native-id']

    #For some reason, attributes are parsed into a list in the HLS metadata. This reformats it into a dictionary.
    attributes_list = metadata['render_dict']['umm']['AdditionalAttributes']

    attributes = {attr['Name']: attr['Values'][0] for attr in attributes_list}
    #print(attributes['MGRS_TILE_ID'])

    if(int(attributes['CLOUD_COVERAGE']) > 50): #Reject granules with large cloud cover, for now
        continue
    time = attributes['SENSING_TIME']
    tile_folder = attributes['MGRS_TILE_ID']
    if specific_tile and not tile_folder == tile:
        continue
    ## ======= Create file directory, if needed  ======= ##
    tile_path = os.path.join(folder_path,tile_folder)
    if not os.path.isdir(tile_path):
         os.mkdir(tile_path)
    folder_name = (f'{name}')
    file_path = os.path.join(tile_path,folder_name)
    if not os.path.isdir(file_path):
        os.mkdir(file_path) #Make folder for granule 

    ## ======= download granule ======= ##
    with open(os.devnull, 'w') as f, redirect_stdout(f):
        downloadPath = earthaccess.download(granule, local_path=file_path)
    downloaded = downloaded + 1
    print(f'{name}')
    if downloaded > num_to_download:
         break

    ## ======= write metadata csv ======= ##
    csv_file = os.path.join(file_path, (f'{folder_name}_metadata.csv'))
    metadata_full_dict = {**attributes, **meta}
    vis_urls = granule.dataviz_links()
    metadata_full_dict['data_vis_url'] = vis_urls[0]
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(metadata_full_dict.keys())
        writer.writerow(metadata_full_dict.values())

QUEUEING TASKS | : 100%|██████████| 18/18 [00:00<00:00, 1657.10it/s]
PROCESSING TASKS | :   0%|          | 0/18 [00:00<?, ?it/s]

PROCESSING TASKS | : 100%|██████████| 18/18 [00:16<00:00,  1.09it/s]
COLLECTING RESULTS | : 100%|██████████| 18/18 [00:00<00:00, 53506.36it/s]
QUEUEING TASKS | : 100%|██████████| 18/18 [00:00<00:00, 1999.93it/s]
PROCESSING TASKS | :   0%|          | 0/18 [00:00<?, ?it/s]

HLS.S30.T11SKU.2018007T184739.v2.0


PROCESSING TASKS | : 100%|██████████| 18/18 [00:13<00:00,  1.34it/s]
COLLECTING RESULTS | : 100%|██████████| 18/18 [00:00<00:00, 90200.09it/s]
QUEUEING TASKS | : 100%|██████████| 15/15 [00:00<00:00, 1580.49it/s]
PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

HLS.S30.T10SGD.2018007T184739.v2.0


...

HLS.L30.T10SGD.2018362T183423.v2.0


PROCESSING TASKS | : 100%|██████████| 18/18 [00:12<00:00,  1.49it/s]
COLLECTING RESULTS | : 100%|██████████| 18/18 [00:00<00:00, 123969.58it/s]
QUEUEING TASKS | : 100%|██████████| 18/18 [00:00<00:00, 1762.35it/s]
PROCESSING TASKS | :   0%|          | 0/18 [00:00<?, ?it/s]

HLS.S30.T11SKU.2018362T184801.v2.0


PROCESSING TASKS | : 100%|██████████| 18/18 [00:12<00:00,  1.41it/s]
COLLECTING RESULTS | : 100%|██████████| 18/18 [00:00<00:00, 94371.84it/s]

HLS.S30.T10SGD.2018362T184801.v2.0
